# Injury Probability Model - Paper Section 4

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score, cross_val_predict, StratifiedShuffleSplit
import xgboost as xgb
import os
os.chdir('..')

### Load Data - Preprocessed in other notebooks

In [ ]:
data=pd.read_csv('data/injury_data/injury_features.csv')

In [ ]:
X = data[['team_id','goal_diff','mins_played', 'matches_played', 'num_tackles', 'num_fouls', 'num_bad_touches', 'num_ball_touches', 'num_dribbles',\
       'num_tackleds', 'num_fouleds', 'days_since_last_game', 'dist_covered', 'metres_per_min', 'hir_dist',\
       'sprint_dist', 'num_hirs', 'num_sprints', 'accels', 'decels',\
       'LI_accels', 'LI_decels', 'acute_workload', 'chronic_workload', 'ACWR', 'num_injuries', 'total_days_out','total_games_missed',\
       'days_out_last_injury','frequency_most_prominent_injury', 'days_out_most_prominent_injury',\
       'days_out_most_serious_injury','injuries_past_three_months',\
       'injuries_past_six_months', 'injuries_past_twelve_months','opp_num_tackles', 'opp_num_fouls','opp_num_bad_touches', 'opp_num_touches',\
       'opp_num_dribbles','opp_num_times_tackled', 'opp_num_times_fouled','opp_matches_played', 'opp_days_since_last_game','attendance',\
       'temp', 'snow', 'windspeedMiles',\
       'weatherCode', 'precipMM', 'humidity', 'visibility', 'pressure',\
       'distance', 'height', 'age', 'rolling_mins_played_exp','days_diff','rolling_days_diff_exp','opp_team_id']]

#X = data[['num_injuries','total_days_out','precipMM','temp','injuries_past_twelve_months','injuries_past_three_months']]
X['total_days_out'] = (X['total_days_out'] / X['num_injuries']).fillna(0)

X_team = data['team_id']
X_opp_team = data['opp_team_id']
y = data['injured'].astype(int)
X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False, random_state=1)
ratio = sum(y_train) / len(y_train)
ratio = np.full(len(y_test),ratio) 

from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_df)
X_test_scaled = scaler.transform(X_test_df)
X_train = pd.DataFrame(X_train_scaled, index=X_train_df.index, columns=X_train_df.columns)
X_test = pd.DataFrame(X_test_scaled, index=X_test_df.index, columns=X_test_df.columns)
teams = X_team.unique()
num_teams = len(teams)

def team_ohe(teams,team_id):
    return teams==team_id

### Train and save the Injury Model w/ Predictions

In [ ]:
X_kfold = data[['team_id','dist_covered', 'acute_workload', 'chronic_workload', 'num_injuries', 'total_days_out','num_dribbles',\
       'days_out_last_injury','frequency_most_prominent_injury', 'days_out_most_prominent_injury',\
       'days_out_most_serious_injury','days_since_last_injury',\
       'injuries_past_twelve_months','opp_num_tackles', 'opp_num_fouls','temp', 'precipMM',
       'distance', 'age', 'opp_team_id']]

X_kfold_wo_nans = X_kfold.fillna(1000)
#X_kfold[['team_id','opp_team_id']] = X_kfold[['team_id','opp_team_id']].astype("category")
#X_kfold_wo_nans[['team_id','opp_team_id']] = X_kfold_wo_nans[['team_id','opp_team_id']].astype("category")

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
preds_all = np.array([])
indexes = np.array([])
model_log_losses = []
model_cis = []

classifiers = [
    #KNeighborsClassifier(1000),
    #LogisticRegression(max_iter=1000,C=0.2),
    #RandomForestClassifier(max_depth=4, n_estimators=300,min_samples_leaf=1),
    XGBClassifier(tree_method='hist', objective='binary:logistic', max_depth=4, n_estimators=400,eta=0.025,colsample_bytree=0.3,colsample_bylevel=0.3, alpha=10)
    #GradientBoostingClassifier(n_estimators=400,max_depth=4,learning_rate=learning_rate)
    #MLPClassifier(alpha=1, max_iter=1000)
]

names = [
    #"Nearest Neighbors",
    #"Logistic Regression",
    #"Random Forest",
    "XGBoost",
    #"Neural Net",
]

param = {'tree_method':'hist', 'objective':'binary:logistic', 'max_depth':4, 'n_estimators':400,'eta':0.025,'colsample_bytree':0.3,'colsample_bylevel':0.3, 'alpha':10}

prediction_dfs = []
train_prediction_dfs = []

index_sets = []
train_index_sets = []
for j in range(len(names)):
    print("Name: ", names[j])
    log_losses= []
    train_log_losses = []
    ratio_lls = []
    for i, (train_index, test_index) in enumerate(kfold.split(X_kfold, y)):
        print("Fold: ", i)
        index_sets.append(test_index)
        train_index_sets.append(train_index)
        indexes = np.append(indexes,test_index)
        scaler = StandardScaler()
        if names[j] == 'XGBoost':
            X_ktrain = X_kfold.iloc[train_index] 
            X_ktest = X_kfold.iloc[test_index] 
        else:
            X_ktrain = X_kfold_wo_nans.iloc[train_index] 
            X_ktest = X_kfold_wo_nans.iloc[test_index] 
        X_ktrain = scaler.fit_transform(X_ktrain)
        X_ktest = scaler.transform(X_ktest)
        filename = 'injury_scaler_efficient.sav'
        pickle.dump(scaler, open(filename, 'wb'))
        y_ktrain = y.iloc[train_index]
        y_ktest = y.iloc[test_index]
        evalset = [(X_ktrain, y_ktrain), (X_ktest,y_ktest)]
        #model = classifiers[j]#KNeighborsClassifier(1000)#LogisticRegression()#XGBClassifier(objective='binary:logistic', max_depth=4, n_estimators=250,learning_rate=0.025,colsample_bytree=0.2)
        #model.fit(X_ktrain, y_ktrain)
        dtrain = xgb.DMatrix(X_ktrain,label=y_ktrain)
        dtest = xgb.DMatrix(X_ktest,label=y_ktest)
        model = xgb.train(param,dtrain,400,[(dtest,'eval'),(dtrain,'train')])
        filename = 'injury_model_efficient.sav'
        pickle.dump(model, open(filename, 'wb'))
        preds = model.predict(dtest)
        preds_training = model.predict(dtrain)
        print(preds)
        #preds_all = np.append(preds_all, preds[:,1].flatten())
        preds_all = np.append(preds_all, preds)
        ratio = np.full(len(y_ktest),y_ktrain.sum()/len(y_ktrain))
        print("Ratio: ", log_loss(y_ktest,ratio))
        ratio_lls.append(log_loss(y_ktest,ratio))
        print(log_loss(y_ktest, preds))
        log_losses.append(log_loss(y_ktest, preds))
        train_log_losses.append(log_loss(y_ktrain,preds_training))
        pred_eval_df = pd.DataFrame([])
        #pred_eval_df['prob'] = preds[:,1].flatten()
        pred_eval_df['prob'] = preds
        #pred_eval_df = pred_eval_df.sort_values('index')
        pred_eval_df['injured'] = y_ktest.reset_index(drop=True)
        pred_eval_df.index = test_index

        train_pred_eval_df = pd.DataFrame([])
        #train_pred_eval_df['prob'] = preds_training[:,1].flatten()
        train_pred_eval_df['prob'] = preds_training
        train_pred_eval_df['injured'] = y_ktrain.reset_index(drop=True)
        train_pred_eval_df.index = train_index
        prediction_dfs.append(pred_eval_df)
        train_prediction_dfs.append(train_pred_eval_df)
print("Mean training log loss: ", np.array(train_log_losses).mean())
print("Mean log loss: ", np.array(log_losses).mean())  
print("STD log loss: ", np.array(log_losses).std())  
model_log_losses.append(np.array(log_losses).mean())
model_cis.append((np.array(log_losses).std() / np.sqrt(10))*1.650)

In [ ]:
data.to_csv('data/predictions/injury_predictions_XGB.csv',index=False)

## Shap Values

In [ ]:
import shap

xgboost_model = XGBClassifier(tree_method="gpu_hist", objective='binary:logistic', max_depth=4, n_estimators=300,eta=0.04,colsample_bytree=0.1,colsample_bylevel=0.3, alpha=10, enable_categorical=True)
xgboost_model.fit(X_ktrain, y_ktrain)
preds = xgboost_model.predict_proba(X_ktest)

def shap_explainer(inj_model,is_tree,train_x,train_y,test_x,index,columns):
    if is_tree:
        explainer = shap.TreeExplainer(inj_model, model_output='probability', feature_dependence='dependent', data=train_x,feature_names=columns)
        shap_values = explainer.shap_values(test_x)
    else:
        explainer = shap.Explainer(inj_model, train_x, feature_names=columns,model_output="probability")
        shap_values = explainer.shap_values(test_x)
    shap.initjs()
    display(shap.force_plot(explainer.expected_value, shap_values, scaler.inverse_transform(test_x[index:index+1000]),feature_names=columns))
    shap_df = pd.DataFrame(data=[(shap_values[0]*100).round(3),scaler.inverse_transform(X_ktest[index:index+1]).flatten().round(1)], columns=columns, index=['% Contribution','Value']).transpose().sort_values(by='% Contribution', ascending=True)

    print("Prob: ", shap_df['% Contribution'].sum().round(4),"%")
    print("Base value: ", (explainer.expected_value*100).round(3), "%")
    print("Total prob: ", (explainer.expected_value*100 + (shap_df['% Contribution'].sum().round(4))).round(3), "%")
    return shap_df, shap_values,explainer

In [ ]:
shap_df, shap_values,explainer = shap_explainer(xgboost_model, True, X_ktrain, y_ktrain, X_ktest,155, X_kfold.columns)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
shap.summary_plot(explainer(X_ktest), plot_type="bar", show=False,plot_size=[10,8])

# Show the plot
plt.xlabel('Mean |SHAP| (average impact on model output)')
plt.show()